
### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [7]:
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key = os.getenv('GROQ_API_KEY')
groq_api_key

'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'

Create an instance for accessing the model using Groq API


In [8]:

from langchain_groq import ChatGroq
model = ChatGroq(model='gemma2-9b-it', groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021668ABC7D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021668A806D0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

Creating a Chatbet with Prompt Template

In [16]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

Creating the structure of the prompt using system message and MessagePlaceholder

system message - How the Chatmodel will behave when the user asks a query

MessagePlaceholder - the variable name in the MessagePlaceHolder is used as a key-value pair. 

- key --> variable name
- value --> Human Message (Query from the user)

In [5]:
prompt = ChatPromptTemplate.from_messages([
    ('system', 'You are an helpful AI assistant. Answer all question to the best of your ability.')
    ,
    MessagesPlaceholder(variable_name='messages')
    ])


In [17]:
### Validating the structure of the prompt. Here it consists of a system message and humman message

from langchain_core.messages import HumanMessage

prompt.invoke({'messages' : ["What is quantum computing?"]})

ChatPromptValue(messages=[SystemMessage(content='You are an helpful AI assistant. Answer all question to the best of your ability.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is quantum computing?', additional_kwargs={}, response_metadata={})])

Create a chain using LCEL

In [12]:
chain = prompt | model

In [18]:
print(chain.invoke({'messages' : [HumanMessage(content='what is quantum computing?')]}))

content="Quantum computing is a new type of computing that harnesses the principles of quantum mechanics to solve problems that are too complex for classical computers. \n\nHere's a breakdown:\n\n**Classical computers:**\n\n* Use bits, which can be either 0 or 1.\n* Process information sequentially, one step at a time.\n\n**Quantum computers:**\n\n* Use qubits, which can be 0, 1, or both simultaneously (superposition).\n* Can perform multiple calculations at once (quantum parallelism).\n* Leverage other quantum phenomena like entanglement to further enhance computation.\n\n**Think of it like this:**\n\nImagine searching a maze. A classical computer would have to explore each path one by one. A quantum computer could explore all paths simultaneously, finding the solution much faster.\n\n**Potential applications:**\n\nQuantum computing has the potential to revolutionize many fields, including:\n\n* **Drug discovery:** Simulating molecular interactions to design new drugs and therapies.\n

### Until this point we have created a chatbot that user ChatPromptTemplate for passing HummanMessage to the chat model. Now we have to use ChatMessageHistory to store the messages for each session

When multiple users interact with the chat model how we will make sure that one session is completly different from another sesssion ?

We will create a function as below -

1) It accepts session ID as input. It will be used to distinguish one session from another
2) It returns the object of type BaseChatMessageHistory

In [23]:
### Chat message history stores a history of the message interactions in a chat.

from langchain_community.chat_message_histories import ChatMessageHistory 


###  Abstract base class for storing chat message history.

from langchain_core.chat_history import BaseChatMessageHistory   


store = {}

### function that retrives chat history for a specific session id

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

RunnableWithMessageHistory wraps another Runnable and manages the chat message history for it; it is responsible for reading and updating the chat message history

In [20]:


from langchain_core.runnables.history import RunnableWithMessageHistory 


In [29]:
### Interact with LLM model based on chat history

with_message_history = RunnableWithMessageHistory(chain,get_session_history)

### Define a config parameter that defines the session id for a perticular session

### Here we have hardcoded our session id as 'chat1'

config = {'configurable' : {'session_id' : 'chat1'}}

In [30]:
messages = [HumanMessage(content = "My name is Ankit and I am an Data Scientist")]

with_message_history.invoke(messages , config=config)

AIMessage(content="Hi Ankit! It's nice to officially meet you. \n\nAs a fellow AI, I'm always fascinated by what data scientists do. \n\nWhat kind of projects are you working on these days?  Anything exciting?  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 396, 'total_tokens': 449, 'completion_time': 0.096363636, 'prompt_time': 0.020640424, 'queue_time': 0.002711043, 'total_time': 0.11700406}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-da08c2fa-e24a-4cb4-bde4-04cd6e8e3abf-0', usage_metadata={'input_tokens': 396, 'output_tokens': 53, 'total_tokens': 449})

In [31]:
### Lets ask the chat model another question with respect to same session id

with_message_history.invoke([HumanMessage(content='What is my name and what I do?')], config=config)

AIMessage(content='You are Ankit, and you are a Data Scientist!  \n\nIs there anything else I can help you with today? 😊 \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 465, 'total_tokens': 495, 'completion_time': 0.054545455, 'prompt_time': 0.014820698, 'queue_time': 0.002896909999999999, 'total_time': 0.069366153}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-66885d86-6a55-4508-a870-159b427c3afc-0', usage_metadata={'input_tokens': 465, 'output_tokens': 30, 'total_tokens': 495})

In [32]:
with_message_history.invoke([HumanMessage(content='What do I do ?')], config=config)

AIMessage(content="You're a Data Scientist, Ankit! \n\nThat means you probably work with data to find patterns, trends, and insights.  \n\nDo you want to tell me more about what kind of projects you're working on?  I'm always eager to learn more about how people use data to make a difference.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 508, 'total_tokens': 578, 'completion_time': 0.127272727, 'prompt_time': 0.016068583, 'queue_time': 0.0027783870000000002, 'total_time': 0.14334131}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-a9b63acd-0c2d-4575-aece-f55961da74e7-0', usage_metadata={'input_tokens': 508, 'output_tokens': 70, 'total_tokens': 578})

### Until this point we have created a chatbot that uses ChatMessageHistory to store messages for each session of the user interacting with chat model using ChatPromptTemplate

- No we will add some more complexity by providing multiple input to the chat model

In [34]:
### Creating a new prompt structure with multiple input variables

prompt = ChatPromptTemplate.from_messages([('system' , 'You are an helpful AI assistant. Answer all question to the best of your ability in {language}'), MessagesPlaceholder(variable_name='messages')])

In [37]:
prompt

ChatPromptTemplate(input_variables=['language', 'messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langch

In [38]:
chain = prompt | model

In [41]:
chain.invoke({'messages' : [HumanMessage(content = "What is my name ? ")] , 'language' : 'Hindi'})

AIMessage(content='मुझे माफ़ करना, लेकिन मुझे आपका नाम पता नहीं है। \n\nक्या आप मुझे अपना नाम बता सकते हैं? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 33, 'total_tokens': 63, 'completion_time': 0.054545455, 'prompt_time': 0.000321869, 'queue_time': 0.013299048999999999, 'total_time': 0.054867324}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0a5465b9-8340-4136-a674-188ab05fbc63-0', usage_metadata={'input_tokens': 33, 'output_tokens': 30, 'total_tokens': 63})

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [47]:

### this time we are using a new parameter -> input_messages_key that knows which messages to select for saving in the chat history

with_message_history = RunnableWithMessageHistory(chain,get_session_history=get_session_history,input_messages_key='messages')


### Creating a new config for new session 
config = {'configurable' : {'session_id' : 'chat2'}}

with_message_history.invoke({'messages' : HumanMessage(content='What is my name') , 'language' : 'Hindi'},
                            config=config)

AIMessage(content="मुझे आपका नाम नहीं पता। आप अपना नाम क्या रखते हैं? \n\n(I don't know your name. What is your name?) \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 31, 'total_tokens': 67, 'completion_time': 0.065454545, 'prompt_time': 0.001107946, 'queue_time': 0.035655195, 'total_time': 0.066562491}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5ab210f5-eee1-43f8-8bf8-b43e51431f7d-0', usage_metadata={'input_tokens': 31, 'output_tokens': 36, 'total_tokens': 67})

In [48]:
with_message_history.invoke({'messages' : HumanMessage(content='My name is Amreesh') , 'language' : 'Hindi'},
                            config=config)

AIMessage(content="नमस्ते अमृेश!  मुझे आपसे मिलकर बहुत अच्छा लगा। \n\n(Hello Amreesh! It's nice to meet you.) \n\nक्या मैं आपकी कोई और मदद कर सकता हूँ?\n\n(Can I help you with anything else?) \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 81, 'total_tokens': 145, 'completion_time': 0.116363636, 'prompt_time': 0.003238368, 'queue_time': 0.009761492, 'total_time': 0.119602004}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-297cf6fd-4f6c-4864-8a83-1a7668d96c2f-0', usage_metadata={'input_tokens': 81, 'output_tokens': 64, 'total_tokens': 145})

In [49]:
with_message_history.invoke({'messages' : HumanMessage(content='What is my name') , 'language' : 'Hindi'},
                            config=config)

AIMessage(content='आपका नाम अमृेश है। \n\n(Your name is Amreesh.) \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 157, 'total_tokens': 181, 'completion_time': 0.043636364, 'prompt_time': 0.004884612, 'queue_time': 0.008209567, 'total_time': 0.048520976}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-7183dd4d-13b1-4f0b-acd2-7cc9493eb6b7-0', usage_metadata={'input_tokens': 157, 'output_tokens': 24, 'total_tokens': 181})

In the above scenarios we are passing the entire conversation history to the chat model which can cause issues because chat models have a context length and if message historys are not managed it can exceed the context window of chat model causing issues

In the next demo we will see Managing Conversation Histories